# Datenbanksetup

Das Notebook dient dazu, das Schema für die GraphDB zu erstellen und die Spielwelt zu generieren.

In [ ]:
import os
from dotenv import load_dotenv
from neo4j import GraphDatabase
from sentence_transformers import SentenceTransformer, util

load_dotenv(dotenv_path='../.env')


In [ ]:
driver = GraphDatabase.driver(
    uri=os.getenv('NEO4J_URI'),
    auth=(
        os.getenv('NEO4J_USER'), 
        os.getenv('NEO4J_PASSWORD')
    )
)

print('Docker starten ;-)')
print(f'URI: {os.getenv("NEO4J_URI")}')
print(f'UI:  http://localhost:7474')

# Queryhelper
def run_query(query, params=None):
    
    # Ergebnisliste
    results = []

    # Session öffnen
    with driver.session() as session:

        # Query
        result = session.run(query, params or {})

        # Ergebnisse wegspeichern
        records = [r.data() for r in result]
        results.extend(records)

    return results

In [ ]:
# WARNUNG: Löscht ALLE Daten!
# Uncomment nur wenn du wirklich resetten willst:

def reset_db():
    with driver.session() as session:
        session.run("MATCH (n) DETACH DELETE n")
        print("✓ Database reset - alle Nodes gelöscht")

reset_db()

In [ ]:
emb_model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')

## Constrains und Index anlegen

In [ ]:
# Constrains festlegen
constrain_queries = [
    'CREATE CONSTRAINT location_id IF NOT EXISTS FOR (l:Location) REQUIRE l.id IS UNIQUE',
    'CREATE CONSTRAINT item_id     IF NOT EXISTS FOR (i:Item)     REQUIRE i.id IS UNIQUE',
    'CREATE CONSTRAINT npc_id      IF NOT EXISTS FOR (n:NPC)      REQUIRE n.id IS UNIQUE',
    'CREATE CONSTRAINT player_id   IF NOT EXISTS FOR (p:Player)   REQUIRE p.id IS UNIQUE',
]

[run_query(query) for query in constrain_queries]
print()

# Index festlegen
entity_indexes = [
    "CREATE INDEX location_name IF NOT EXISTS FOR (l:Location) ON (l.name)",
    "CREATE INDEX item_name     IF NOT EXISTS FOR (i:Item)     ON (i.name)"
]

[run_query(query) for query in entity_indexes]

## Objekte erstellen

In [ ]:
# Orte/Räume: id, name, beschreibung
locations_data = [
    {
        'id': 'taverne',
        'name': 'Mo\'s Taverne',
        'description': 'Eine alte Taverne, etwas heruntergekommen aber bleibt. Hier wird jeden Abend gefeiert.',
    },
    {
        'id': 'marktplatz',
        'name': 'Marktplatz',
        'description': 'Das ist der Markplatz des kleinen, beschaulichen Ortes.'
    },
    {
        'id': 'finsterwald',
        'name': 'Finsterwald',
        'description': 'in dunkler, übelriechender Wald voller Geister und anderen Ängsten.'
    },

]


for location in locations_data:

    query = """
    MERGE (l:Location {id: $id})
    ON CREATE SET 
        l.name = $name, 
        l.description = $description,
        l.name_emb = $name_emb,
        l.description = $description_emb
    RETURN l
    """

    name_emb = emb_model.encode(location['name'])
    description_emb = emb_model.encode(location['description'])

    params = {
        'id': location['id'],
        'name': location['name'],
        'description': location['description'],
        'name_emb': name_emb,
        'description_emb': description_emb
    }

    run_query(query, params)

In [ ]:
# Items: id, name, beschreibung
items_data = [
    {
        'id': 'truhe',
        'name': 'Alte Truhe',
        'description': 'Eine alte Truhe mit einem rostigen Schloss. Sie steht schon länger hier. Alt und verwittert.'
    },
    {
        'id': 'schluessel',
        'name': 'Rostiger Schlüssel',
        'description': 'Ein rostiger Schlüssel der nur in ein ganz bestimmtes Schloss passt.'
    }
]


for item in items_data:

    query = """
    MERGE (l:Item {id: $id})
    ON CREATE SET 
        l.name = $name, 
        l.description = $description,
        l.name_emb = $name_emb,
        l.description = $description_emb
    RETURN l
    """

    name_emb = emb_model.encode(location['name'])
    description_emb = emb_model.encode(location['description'])

    params = {
        'id': item['id'],
        'name': item['name'],
        'description': item['description'],
        'name_emb': name_emb,
        'description_emb': description_emb
    }

    run_query(query, params)

In [ ]:
# NPCs: id, name, beschreibung
persons_data = [
    {
        'id': 'wirt',
        'name': 'Schenk',
        'description': 'Ein alter, grummiger Wirt, der seinen Gästen stets zu wenig einschenkt.'
    },
    {
        'id': 'player',
        'name': 'Player',
        'description': 'Hier könnte Dein Name stehen!'
    }
]

for person in persons_data:

    query = """
    MERGE (l:Persons {id: $id})
    ON CREATE SET 
        l.name = $name, 
        l.description = $description,
        l.name_emb = $name_emb,
        l.description = $description_emb
    RETURN l
    """
    name_emb = emb_model.encode(location['name'])
    description_emb = emb_model.encode(location['description'])

    params = {
        'id': person['id'],
        'name': person['name'],
        'description': person['description'],
        'name_emb': name_emb,
        'description_emb': description_emb
    }

    run_query(query, params)

## Placing aller Objekte

In [ ]:
# Persons start
player_edges = [
    """
    MATCH (start:Persons {id: 'player'}), (ende:Location {id: 'marktplatz'})
    MERGE (start)-[:IST_IN]->(ende)
    RETURN start, ende
    """,
    """
    MATCH (start:Persons {id: 'wirt'}), (ende:Location {id: 'taverne'})
    MERGE (start)-[:IST_IN]->(ende)
    RETURN start, ende
    """
]

[run_query(query) for query in player_edges]

In [ ]:
# Wege
location_edges = [
    """
    MATCH 
        (m:Location {id: 'marktplatz'}),
        (t:Location {id: 'taverne'}),
        (f:Location {id: 'finsterwald'})
    MERGE (m)-[:ERREICHT]->(t)
    MERGE (t)-[:ERREICHT]->(m)
    MERGE (m)-[:ERREICHT]->(f)
    MERGE (f)-[:ERREICHT]->(m)
    """
]

[run_query(query) for query in location_edges]

In [ ]:
# Items
item_edges = [
    """
    MATCH 
        (ta:Location {id: 'taverne'}), 
        (f:Location {id: 'finsterwald'}),
        (s:Item {id: 'schluessel'}),
        (tr:Item {id: 'truhe'})
    MERGE (s)-[:IST_IN]->(ta)
    MERGE (tr)-[:IST_IN]->(f)
    """
]

[run_query(query) for query in item_edges]

In [ ]:
# Quests
quest_edges = [
    """
    MATCH
        (s:Item {id: 'schluessel'}), 
        (t:Item {id: 'truhe'})
    MERGE (s)-[:OEFFNET]->(t)
    """
]

[run_query(query) for query in quest_edges]